## Pytroch Version

In [1]:
import torch.nn as nn

In [2]:
class ZFNet(nn.Module):
    
    def __init__(self):
        super(ZFNet, self).__init__()
        self.features = nn.Sequential(
                        nn.Conv2d(3, 96, 7, 2, 3),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2, 1),
                        nn.LocalResponseNorm(2),
                        nn.Conv2d(96, 256, 5, 2, 2),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2, 1),
                        nn.LocalResponseNorm(2),
                        nn.Conv2d(256, 384, 3, 1, 1),
                        nn.ReLU(),
                        nn.Conv2d(384, 384, 3, 1, 1),
                        nn.ReLU(),
                        nn.Conv2d(384, 256, 3, 1, 1),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2)
        )
        self.classifier = nn.Sequential(
                        nn.Linear(6 * 6 * 256, 4096),
                        nn.Dropout(0.5),
                        nn.Linear(4096, 4096),
                        nn.Dropout(0.5),
                        nn.Linear(4096, 1000)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 6 * 6 * 256)
        x = self.classifier(x)
        return x



In [3]:
net = ZFNet()
net

ZFNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (7): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): Dropout(p=0.5)
    (

## TensorFlow Version


In [4]:
import tensorflow as tf

In [6]:
x = tf.placeholder(tf.float32, [None, 224, 224, 3], name = 'Input')

W_conv = {
    'conv1': tf.Variable(tf.truncated_normal([7, 7, 3, 96])),
    'conv2': tf.Variable(tf.truncated_normal([5, 5, 96, 256])),
    'conv3': tf.Variable(tf.truncated_normal([3, 3, 256, 384])),
    'conv4': tf.Variable(tf.truncated_normal([3, 3, 384, 384])),
    'conv5': tf.Variable(tf.truncated_normal([3, 3, 384, 256])),
    'fc6': tf.Variable(tf.truncated_normal([9216, 4096])),
    'fc7': tf.Variable(tf.truncated_normal([4096, 4096])),
    'fc8': tf.Variable(tf.truncated_normal([4096, 1000]))
}

b_conv = {
    'conv1': tf.Variable(tf.constant(0.0, shape=[96])),
    'conv2': tf.Variable(tf.constant(0.0, shape=[256])),
    'conv3': tf.Variable(tf.constant(0.0, shape=[384])),
    'conv4': tf.Variable(tf.constant(0.0, shape=[384])),
    'conv5': tf.Variable(tf.constant(0.0, shape=[256])),
    'fc6': tf.Variable(tf.constant(0.1, shape=[4096])),
    'fc7': tf.Variable(tf.constant(0.1, shape=[4096])),
    'fc8': tf.Variable(tf.constant(0.1, shape=[1000]))
    
}

print('注意：ZFNet模型结构存在计算错误')
print()
conv1 = tf.nn.conv2d(x, W_conv['conv1'], strides=[1, 2, 2, 1], padding='VALID')
conv1 = tf.nn.bias_add(conv1, b_conv['conv1'])
conv1 = tf.nn.relu(conv1)
print('conv1:', conv1.shape)
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
print('pool1:', pool1.shape)
norm1 = tf.nn.lrn(pool1)
print('norm:', norm1.shape)

conv2 = tf.nn.conv2d(norm1, W_conv['conv2'], strides=[1, 2, 2, 1], padding='VALID')
conv2 = tf.nn.bias_add(conv2, b_conv['conv2'])
conv2 = tf.nn.relu(conv2)
print('conv2:', conv2.shape)
pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
print('pool2:', pool2.shape)
norm2 = tf.nn.lrn(pool2)
print('norm2:', norm2.shape)

conv3 = tf.nn.conv2d(norm2, W_conv['conv3'], strides=[1, 1, 1, 1], padding='SAME')
conv3 = tf.nn.bias_add(conv3, b_conv['conv3'])
conv3 = tf.nn.relu(conv3)
print('conv3:', conv3.shape)

conv4 = tf.nn.conv2d(conv3, W_conv['conv4'], strides=[1, 1, 1, 1], padding='SAME')
conv4 = tf.nn.bias_add(conv4, b_conv['conv4'])
conv4 = tf.nn.relu(conv4)
print('conv4:', conv4.shape)

conv5 = tf.nn.conv2d(conv4, W_conv['conv5'], strides=[1, 1, 1, 1], padding='SAME')
conv5 = tf.nn.bias_add(conv5, b_conv['conv5'])
conv5 = tf.nn.relu(conv5)
print('conv5:', conv5.shape)
pool3 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')
print('pool3:', pool3.shape)

reshape = tf.reshape(pool3, [-1, 6 * 6 * 256])
fc6 = tf.add(tf.matmul(reshape, W_conv['fc6']), b_conv['fc6'])
fc6 = tf.nn.relu(fc6)
fc6 = tf.nn.dropout(fc6, 0.5)
print('fc6:', fc6.shape)

fc7 = tf.add(tf.matmul(fc6, W_conv['fc7']), b_conv['fc7'])
fc7 = tf.nn.relu(fc7)
fc7 = tf.nn.dropout(fc7, 0.5)
print('fc7:', fc7.shape)

output = tf.nn.softmax(tf.add(tf.matmul(fc7, W_conv['fc8']), b_conv['fc8']))
print('output:', output.shape)



注意：ZFNet模型结构存在计算错误

conv1: (?, 109, 109, 96)
pool1: (?, 54, 54, 96)
norm: (?, 54, 54, 96)
conv2: (?, 25, 25, 256)
pool2: (?, 12, 12, 256)
norm2: (?, 12, 12, 256)
conv3: (?, 12, 12, 384)
conv4: (?, 12, 12, 384)
conv5: (?, 12, 12, 256)
pool3: (?, 5, 5, 256)
fc6: (?, 4096)
fc7: (?, 4096)
output: (?, 1000)
